![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/HuggingFace%20in%20Spark%20NLP%20-%20XlmRobertaForQuestionAnswering.ipynb)

## Import XlmRoBertaForQuestionAnswering models from HuggingFace 🤗 into Spark NLP 🚀 

Let's keep in mind a few things before we start 😊 

- This feature is only in `Spark NLP 4.0.0` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import XLM-RoBERTa models trained/fine-tuned for question answering via `XLMRobertaForQuestionAnswering` or `TFXLMRobertaForQuestionAnswering`. These models are usually under `Question Answering` category and have `xlm-roberta` in their labels
- Reference: [TFXLMRobertaForQuestionAnswering](https://huggingface.co/transformers/model_doc/xlm-roberta#transformers.TFXLMRobertaForQuestionAnswering)
- Some [example models](https://huggingface.co/models?filter=xlm-roberta&pipeline_tag=question-answering)

## Export and Save HuggingFace model

- Let's install `HuggingFace` and `TensorFlow`. You don't need `TensorFlow` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock TensorFlow on `2.11.0` version and Transformers on `4.39.3`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.
- XLM-RoBERTa uses SentencePiece, so we will have to install that as well


In [ ]:
!pip install -q transformers==4.39.3 tensorflow==2.11.0 sentencepiece

- HuggingFace comes with a native `saved_model` feature inside `save_pretrained` function for TensorFlow based models. We will use that to save it as TF `SavedModel`.
- We'll use [deepset/xlm-roberta-base-squad2](https://huggingface.co/deepset/xlm-roberta-base-squad2) model from HuggingFace as an example
- In addition to `TFXLMRobertaForQuestionAnswering` we also need to save the `XLMRobertaTokenizer`. This is the same for every model, these are assets needed for tokenization inside Spark NLP.

In [ ]:
from transformers import TFXLMRobertaForQuestionAnswering, XLMRobertaTokenizer 
import tensorflow as tf

MODEL_NAME = 'deepset/xlm-roberta-base-squad2'

tokenizer = XLMRobertaTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained('./{}_tokenizer/'.format(MODEL_NAME))

try:
  model = TFXLMRobertaForQuestionAnswering.from_pretrained(MODEL_NAME)
except:
  model = TFXLMRobertaForQuestionAnswering.from_pretrained(MODEL_NAME, from_pt=True)
    
# Define TF Signature
@tf.function(
  input_signature=[
      {
          "input_ids": tf.TensorSpec((None, None), tf.int32, name="input_ids"),
          "attention_mask": tf.TensorSpec((None, None), tf.int32, name="attention_mask")
      }
  ]
)
def serving_fn(input):
    return model(input)

model.save_pretrained("./{}".format(MODEL_NAME), saved_model=True, signatures={"serving_default": serving_fn})

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/605 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFXLMRobertaForQuestionAnswering: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFXLMRobertaForQuestionAnswering from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLMRobertaForQuestionAnswering from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFXLMRobertaForQuestionAnswering were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaForQuestionAnswering for predictions without further training.


INFO:tensorflow:Assets written to: ./deepset/xlm-roberta-base-squad2/saved_model/1/assets


INFO:tensorflow:Assets written to: ./deepset/xlm-roberta-base-squad2/saved_model/1/assets


Let's have a look inside these two directories and see what we are dealing with:

In [ ]:
!ls -l {MODEL_NAME}

total 2197848
-rw-r--r--  1 maziyar  staff         787 Dec 15 18:30 config.json
drwxr-xr-x  3 maziyar  staff          96 Dec 15 18:29 saved_model
-rw-r--r--  1 maziyar  staff  1110105320 Dec 15 18:30 tf_model.h5


In [ ]:
!ls -l {MODEL_NAME}/saved_model/1

total 18768
drwxr-xr-x  2 maziyar  staff       64 Dec 15 18:29 assets
-rw-r--r--  1 maziyar  staff       55 Dec 15 18:29 fingerprint.pb
-rw-r--r--  1 maziyar  staff   165560 Dec 15 18:30 keras_metadata.pb
-rw-r--r--  1 maziyar  staff  9434510 Dec 15 18:30 saved_model.pb
drwxr-xr-x  4 maziyar  staff      128 Dec 15 18:29 variables


In [ ]:
!ls -l {MODEL_NAME}_tokenizer

total 9920
-rw-r--r--  1 maziyar  staff  5069051 Dec 15 18:28 sentencepiece.bpe.model
-rw-r--r--  1 maziyar  staff      167 Dec 15 18:28 special_tokens_map.json
-rw-r--r--  1 maziyar  staff      699 Dec 15 18:28 tokenizer_config.json


- As you can see, we need the SavedModel from `saved_model/1/` path
- We also be needing `sentencepiece.bpe.model` from the tokenizer
- All we need is to just copy the `sentencepiece.bpe.model` to `saved_model/1/assets` which Spark NLP will look for

In [ ]:
asset_path = '{}/saved_model/1/assets'.format(MODEL_NAME)

!cp {MODEL_NAME}_tokenizer/sentencepiece.bpe.model {asset_path}

Voila! We have our `sentencepiece.bpe.model` inside assets directory

In [ ]:
!ls -l {MODEL_NAME}/saved_model/1/assets

total 9904
-rw-r--r--  1 maziyar  staff  5069051 Dec 15 18:30 sentencepiece.bpe.model


## Import and Save XlmRoBertaForQuestionAnswering in Spark NLP


- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [ ]:
! wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash

Installing PySpark 3.2.1 and Spark NLP 4.2.5
setup Colab for PySpark 3.2.1 and Spark NLP 4.2.5


Let's start Spark with Spark NLP included via our simple `start()` function

In [ ]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

- Let's use `loadSavedModel` functon in `XlmRoBertaForQuestionAnswering` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `XlmRoBertaForQuestionAnswering` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.



In [ ]:
from sparknlp.annotator import *
from sparknlp.base import *

spanClassifier = XlmRoBertaForQuestionAnswering.loadSavedModel(
     '{}/saved_model/1'.format(MODEL_NAME),
     spark
 )\
  .setInputCols(["document_question",'document_context'])\
  .setOutputCol("answer")\
  .setCaseSensitive(False)\
  .setMaxSentenceLength(512)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [ ]:
spanClassifier.write().overwrite().save("./{}_spark_nlp".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [ ]:
!rm -rf {MODEL_NAME}_tokenizer {MODEL_NAME}

Awesome 😎  !

This is your XlmRoBertaForQuestionAnswering model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀 

In [ ]:
! ls -l {MODEL_NAME}_spark_nlp

total 2198320
drwxr-xr-x  3 maziyar  staff          96 Dec 15 18:30 fields
drwxr-xr-x  6 maziyar  staff         192 Dec 15 18:30 metadata
-rw-r--r--  1 maziyar  staff  1119269627 Dec 15 18:31 xlm_roberta_classification_tensorflow
-rw-r--r--  1 maziyar  staff     5069051 Dec 15 18:31 xlmroberta_spp


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny XlmRoBertaForQuestionAnswering model in Spark NLP 🚀 pipeline! 

In [ ]:
document_assembler = MultiDocumentAssembler() \
    .setInputCols(["question", "context"]) \
    .setOutputCols(["document_question", "document_context"])

spanClassifier_loaded = XlmRoBertaForQuestionAnswering.load("./{}_spark_nlp".format(MODEL_NAME))\
  .setInputCols(["document_question",'document_context'])\
  .setOutputCol("answer")

pipeline = Pipeline().setStages([
    document_assembler,
    spanClassifier_loaded
])

example = spark.createDataFrame([["What's my name?", "My name is Clara and I live in Berkeley."]]).toDF("question", "context")
result = pipeline.fit(example).transform(example)

result.select("answer.result").show(1, False)

+-------+
|result |
+-------+
|[clara]|
+-------+



That's it! You can now go wild and use hundreds of `XlmRoBertaForQuestionAnswering` models from HuggingFace 🤗 in Spark NLP 🚀 
